## Lab 3: Model Optimisation

## Exercise 2: Xgboost with Hyperopt

We will train a Xgboost model on the same dataset as previously usiong Hyperopt.


**Pre-requisites:**
- Create a github account (https://github.com/join)
- Install git (https://git-scm.com/book/en/v2/Getting-Started-Installing-Git)
- Install Docker (https://docs.docker.com/get-docker/)

The steps are:
1.   Launch Docker image
2.   Load Data
3.   Train Xgboost model with defauly hyperparameter
4.   Hyperparameter tuning with Hyperopt
5.   Push changes

In [ ]:
"""

# Launch Docker image
docker run  -dit --rm --name adv_dsi_lab_3 -p 8888:8888 -e JUPYTER_ENABLE_LAB=yes -v ~/Projects/adv_dsi/adv_dsi_lab_3:/home/jovyan/work -v ~/.aws:/home/jovyan/.aws -v ~/Projects/adv_dsi/src:/home/jovyan/work/src xgboost-notebook:latest 
docker logs --tail 50 adv_dsi_lab_2

# Create a new git branch called xgboost_hyperopt
git checkout -b xgboost_hyperopt

# Navigate the folder notebooks and create a new jupyter notebook called 2_xgboost_hyperopt.ipynb

"""

## 2. Load Data

**[2.1]** Import the function you created `load_sets` from `src/data/sets`

**[2.2]** Load the saved sets from `data/processed`


In [1]:
# Import the function you created load_sets from src/data/sets
from src.data.sets import load_sets

# Load the saved sets from data/processed
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/')

# 3. Train Xgboost model

**[3.1]** Import the xgboost package as xgb

**[3.2]** Instantiate the RandomForest class into a variable called rf with random_state=8

**[3.3]** Task: Fit the model with the prepared data

**[3.4]** Import `dump` from `joblib` and save the fitted model into the folder `models` as a file called `xgboost_default`

**[3.5]** Save the predictions from this model for the training and validation sets into 2 variables called `y_train_preds` and `y_val_preds`

**[3.6]** Import `print_reg_perf` from `src/models/performance` and display the accuracy and f1 scores of this baseline model on the training and validation sets


In [11]:
# Import the xgboost package as xgb
import xgboost as xgb

# instantiate the XGBClassifier class into a variable called xgboost1
xgboost1 = xgb.XGBClassifier()

# Fit the model with the prepared data
xgboost1.fit(X_train, y_train)
print(xgboost1.fit(X_train, y_train))

# Import dump from joblib and save the fitted model into the folder models as a file called xgboost_default
from joblib import dump 
dump(xgboost1,  '../models/xgboost_default.joblib')

# Save the predictions from this model for the training and validation sets into 2 variables called y_train_preds and y_val_preds
y_train_preds = xgboost1.predict(X_train)
y_val_preds = xgboost1.predict(X_val)

from src.models.performance import print_class_perf

print_class_perf(y_preds=y_train_preds, y_actuals=y_train, set_name='Training', average='weighted')
print_class_perf(y_preds=y_val_preds, y_actuals=y_val, set_name='Validation', average='weighted')

Accuracy Training: 0.9211552214945801
F1 Training      : 0.9208732214873642
Accuracy Validation: 0.9068593424622957
F1 Validation      : 0.9063573849035116


## 4. Hyperparameter tuning with Hyperopt

**[4.1]** Import Trials, STATUS_OK, tpe, hp, fmin from hyperopt package

**[4.2]** Define the search space for xgboost hyperparameters

In [4]:
# Import Trials, STATUS_OK, tpe, hp, fmin from hyperopt package
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

# Define the search space for xgboost hyperparameters
space = {
    'max_depth' : hp.choice('max_depth', range(5, 20, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.05),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.05)
}

**[4.3]** Define a function called `objective` with the following logics:
- input parameters: hyperparameter seacrh space (`space`)
- logics: train a xgboost model with the search space and calculate the average accuracy score for cross validation with 10 folds
- output parameters: dictionary with the loss score and STATUS_OK

In [6]:
# Define a function called objective
def objective(space):
    from sklearn.model_selection import cross_val_score
    
    xgboost = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree']
    )
    
    acc = cross_val_score(xgboost, X_train, y_train, cv=10, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

**[4.4]** Launch Hyperopt search and save the result in a variable called `best`

In [7]:
# Launch Hyperopt search and save the result in a variable called `best`
best = fmin(
    fn=objective,   
    space=space,       
    algo=tpe.suggest,       
    max_evals=5
)

100%|██████████| 5/5 [23:15<00:00, 279.19s/trial, best loss: 0.1020389329003667] 


**[4.5]** Print the best set of hyperparameters

In [8]:
# Print the best set of hyperparameters
print("Best: ", best)

Best:  {'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.25, 'max_depth': 8, 'min_child_weight': 4.0, 'subsample': 0.25}


**[4.6]** Instantiate a XGBClassifier with best set of hyperparameters

**[4.7]** Fit the model with the prepared data

In [9]:
# Instantiate a XGBClassifier with best set of hyperparameters
xgboost2 = xgb.XGBClassifier(
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    min_child_weight = best['min_child_weight'],
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree']
)

# Fit the model with the prepared data
xgboost2.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9500000000000001, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=8,
              min_child_weight=4.0, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.25,
              tree_method='exact', validate_parameters=1, verbosity=None)

**[4.8]** Display the accuracy and f1 scores of this baseline model on the training and validation sets

In [12]:
# Display the accuracy and f1 scores of this baseline model on the training and validation sets
print_class_perf(y_preds=xgboost2.predict(X_train), y_actuals=y_train, set_name='Training', average='weighted')
print_class_perf(y_preds=xgboost2.predict(X_val), y_actuals=y_val, set_name='Validation', average='weighted')

Accuracy Training: 0.9243174264839926
F1 Training      : 0.9240612030551106
Accuracy Validation: 0.9039776193180623
F1 Validation      : 0.9035708202543029


**[4.9]** Save the fitted model into the folder models as a file called `xgboost_best`

In [13]:
# Save the fitted model into the folder models as a file called `xgboost_best`
dump(xgboost2,  '../models/xgboost_best.joblib')

['../models/xgboost_best.joblib']

## 5. Push changes

In [ ]:
"""
# Add you changes to git staging area
# Create the snapshot of your repository and add a description
# Push your snapshot to Github

git add .   
git commit -m "commit 3 xgboost"
git push https://*********@github.com/CazMayhem/adv_dsi_lab_3.git

# Check out to the master branch
# Pull the latest updates

git checkout master
git pull https://*********@github.com/CazMayhem/adv_dsi_lab_3.git

# Check out to the data_prep branch
# Merge the master branch and push your changes
git checkout xgboost_hyperopt
git merge master
git push https://*********@github.com/CazMayhem/adv_dsi_lab_3.git

"""

**[5.8]** Go to Github and merge the branch after reviewing the code and fixing any conflict


**[5.9]** Stop the Docker container

=> docker stop adv_dsi_lab_3